In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('definitions22.csv', encoding='latin1')

# create a boolean indicator =True if the definition is of "machine learning"
data['ml'] = data['term'] == 'machine learning'

print(data.dtypes)
print(data.head(10))

term          object
definition    object
ml              bool
dtype: object
               term                                         definition    ml
0  machine learning                        Machine do the work for me.  True
1  machine learning  The process of teaching a computer over iterat...  True
2  machine learning  Machine learning is using quantitative data cl...  True
3  machine learning  Training machine to understand human's cognition.  True
4  machine learning  the use of algorithms to inform a program of i...  True
5  machine learning  machine learning is the process of analyzing d...  True
6  machine learning                                training to predict  True
7  machine learning  synthetically adjusting future behaviors based...  True
8  machine learning  A series of algorithmic procedures that proces...  True
9  machine learning  The process of artificial programs' iteration ...  True


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

L = data["ml"] # labels
corpus = data['definition'] # corpus of reviews in words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

(57, 358)


## Extract polynomial features

In [5]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2, interaction_only=True)
Xp = poly.fit_transform(X)
Xp.shape

(57, 64262)

## Polynomial features with the kernel trick

In [6]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=123000)
X_features = rbf_feature.fit_transform(X)
# fit SGDClassifier
clf = SGDClassifier(max_iter=5)
clf.fit(X_features, L)   


/Users/aaron/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
X_features.shape

(57, 123000)

In [8]:
import my_measures

performance_measures = my_measures.BinaryClassificationPerformance(clf.predict(X_features), L, 'clf')
performance_measures.compute_measures()
print(performance_measures.performance_measures)

{'Pos': 19, 'Neg': 38, 'TP': 17, 'TN': 38, 'FP': 0, 'FN': 2, 'Accuracy': 0.9649122807017544, 'Precision': 1.0, 'Recall': 0.8947368421052632, 'desc': 'clf'}


In [9]:
# machine learning definitions
# Wikipedia, Expert System, Tech Emergence
ml_defs = ["Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.",
          "Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.",
          "Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions."]
    
# AI definitions
# Wikipedia, Oxford dictionary
ai_defs = ["Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.",
          "the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages"]

# Definitions of unrelated things: kitten, piano, widget
other_defs = ["A kitten, also known as a kitty or kitty cat, is a juvenile cat.",
             "The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.",
             "a small gadget or mechanical device, especially one whose name is unknown or unspecified"]
    
def get_prediction(definition):
    text_x = vectorizer.transform([definition]).toarray()
    rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=123000)
    X_features = rbf_feature.fit_transform(text_x)
    return(clf.predict(X_features))

print("Model predictions for 'machine learning' definitions:")
for mld in ml_defs:
    print(get_prediction(mld))
    
print("* * *")
print("Model predictions for 'AI' definitions:")
for aid in ai_defs:
    print(get_prediction(aid))

print("* * *")
print("Model predictions for other definitions (kitten, piano, widget):")
for otherd in other_defs:
    print(get_prediction(otherd))

Model predictions for 'machine learning' definitions:
[False]
[False]
[False]
* * *
Model predictions for 'AI' definitions:
[False]
[False]
* * *
Model predictions for other definitions (kitten, piano, widget):
[False]
[False]
[False]
